In [ ]:
from ultralytics import YOLO
import os
import shutil
import random

In [ ]:
!mkdir dataset

In [ ]:
!mkdir dataset/labels
!mkdir dataset/images

In [ ]:
!cp '/kaggle/input/large-scale-image-dataset-of-wood-surface-defects/Bounding Boxes - YOLO Format - 1/Bounding Boxes - YOLO Format - 1'/* ./dataset/labels

In [ ]:
!cp '/kaggle/input/large-scale-image-dataset-of-wood-surface-defects/Images - 1/Images - 1'/* ./dataset/images

In [ ]:
!cp /kaggle/input/dtyaml/data.yaml ./dataset/

In [ ]:
# Пути к папкам
dataset_path = "/kaggle/working/dataset"
images_path = os.path.join(dataset_path, "images")
labels_path = os.path.join(dataset_path, "labels")

# Папки для train, val и test
train_images_path = os.path.join(dataset_path, "train", "images")
train_labels_path = os.path.join(dataset_path, "train", "labels")
val_images_path = os.path.join(dataset_path, "val", "images")
val_labels_path = os.path.join(dataset_path, "val", "labels")
test_images_path = os.path.join(dataset_path, "test", "images")
test_labels_path = os.path.join(dataset_path, "test", "labels")

# Создаем папки, если их нет
os.makedirs(train_images_path, exist_ok=True)
os.makedirs(train_labels_path, exist_ok=True)
os.makedirs(val_images_path, exist_ok=True)
os.makedirs(val_labels_path, exist_ok=True)
os.makedirs(test_images_path, exist_ok=True)
os.makedirs(test_labels_path, exist_ok=True)

# Список всех файлов изображений
image_files = [f for f in os.listdir(images_path) if os.path.isfile(os.path.join(images_path, f))]
random.shuffle(image_files)

# Пропорции для train, val и test
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

train_size = int(len(image_files) * train_ratio)
val_size = int(len(image_files) * val_ratio)

# Разделяем файлы на train, val и test
train_files = image_files[:train_size]
val_files = image_files[train_size:train_size + val_size]
test_files = image_files[train_size + val_size:]

# Функция для перемещения файлов
def move_files(file_list, source_dir, target_image_dir, target_label_dir):
    for file_name in file_list:
        base_name = os.path.splitext(file_name)[0]
        image_src = os.path.join(source_dir, file_name)
        label_src = os.path.join(labels_path, base_name + ".txt")
        image_dest = os.path.join(target_image_dir, file_name)
        label_dest = os.path.join(target_label_dir, base_name + ".txt")

        print(f"Copying {image_src} to {image_dest}")
        print(f"Copying {label_src} to {label_dest}")

        if os.path.exists(image_src) and os.path.exists(label_src):
            shutil.copy(image_src, image_dest)
            shutil.copy(label_src, label_dest)

# Перемещаем файлы в соответствующие папки
move_files(train_files, images_path, train_images_path, train_labels_path)
move_files(val_files, images_path, val_images_path, val_labels_path)
move_files(test_files, images_path, test_images_path, test_labels_path)

In [ ]:
model = YOLO('/kaggle/working/runs/detect/train/weights/best.pt')
results = model.train(data='/kaggle/working/dataset/data.yaml',
                      epochs=100,
                      imgsz=1024,
                      batch=4,
                      workers=16)

In [4]:
from ultralytics import YOLO
model = YOLO('/kaggle/working/runs/detect/train/weights/best.pt')
results = model.val(data='/kaggle/working/dataset/data.yaml',
                    imgsz=1024,
                    workers=16)

Ultralytics YOLOv8.2.87 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv10x summary (fused): 503 layers, 31,599,488 parameters, 0 gradients, 169.9 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 33.0MB/s]
val: Scanning /kaggle/working/dataset/val/labels.cache... 800 images, 93 backgrounds, 0 corrupt: 100%|██████████| 800/800 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [00:27<00:00,  1.79it/s]


                   all        800       1791      0.683      0.624      0.656      0.392
             Quartzity         28         33      0.422      0.303      0.309      0.239
             Live_Knot        431        768      0.779      0.786      0.831      0.381
                Marrow         42         45      0.836      0.756      0.803      0.562
                 resin        100        128      0.759      0.665      0.741      0.408
             Dead_Knot        368        568      0.757      0.815      0.842      0.454
       knot_with_crack         79        113      0.677      0.574      0.592      0.403
          Knot_missing         18         19      0.538       0.49      0.499      0.324
                 Crack         76        117      0.697      0.607      0.633      0.366
Speed: 0.5ms preprocess, 29.6ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/val
